In [6]:
import pandas as pd 

from showupforhealth import params, utils
from showupforhealth.ml_functions import preprocessor

NameError: name 'df' is not defined

In [7]:
data = pd.read_csv(f'{OUTPUT_DATA}training_data_no_features.csv')

NameError: name 'OUTPUT_DATA' is not defined

In [8]:
data.head

NameError: name 'data' is not defined